In [ ]:
%cd ..
%load_ext autoreload
%autoreload 2

# Mini EDA

In [ ]:
import polars as pl

In [ ]:
pl.scan_csv('data/train.csv').select(pl.col('store_nbr')).unique().collect()

In [ ]:
pl.scan_csv('data/train.csv').filter(pl.col('store_nbr') == 47).head().collect()

In [ ]:
(pl.scan_csv('data/train.csv')
 .filter(pl.col('store_nbr') == 47)
 .with_columns(pl.col('date').str.to_date(format='%Y-%m-%d'))
 .with_columns(pl.col('date').dt.year().alias('year'))
 .group_by('year')
 .len()
 .collect())

In [ ]:
store47_2016 = (
    pl.scan_csv('data/train.csv')
    .filter(pl.col('store_nbr') == 47)
    .with_columns(
        pl.col('date').str.to_date(format='%Y-%m-%d'),
        pl.col('onpromotion').replace_strict({"False": False, "True": True}))
    .filter(pl.col('date').dt.year().cast(pl.Int32) == 2016)
    .collect()
)

In [ ]:
store47_2016.write_parquet('data/store47_2016.zstd', compression='zstd')

In [ ]:
pl.scan_parquet('data/store47_2016.zstd').head().collect()

In [ ]:
pl.scan_parquet('data/store47_2016.zstd').describe()

In [ ]:
# remove negative unit_sales(
(pl.scan_parquet('data/store47_2016.zstd')
.filter(pl.col('unit_sales') >= 0)
.collect()
.write_parquet('data/store47_2016_positive.zstd', compression='zstd')
)

# Download

In [ ]:
import polars as pl
import pyarrow.dataset as ds

dset = ds.dataset('s3://cd4ml/data/raw/store47_2016_positive.zstd', format='parquet')
(
    pl.scan_pyarrow_dataset(dset)
    .head()
    .collect()
)